In [97]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from PIL import Image
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Define a threshold for contrast labeling
CONTRAST_THRESHOLD = 4.5

def calculate_contrast(image):
    """
    Calculate the contrast ratio between text and background in a grayscale image.
    Returns the contrast ratio.
    """
    grayscale_image = image.convert('L')
    pixel_values = np.array(grayscale_image)

    # Get the min and max intensities (assuming these represent background and text)
    min_intensity = pixel_values.min()
    max_intensity = pixel_values.max()
    
    # Calculate the contrast ratio using WCAG guidelines
    l1 = (max_intensity + 0.05) / 255
    l2 = (min_intensity + 0.05) / 255
    contrast_ratio = (l1 + 0.05) / (l2 + 0.05) if l1 > l2 else (l2 + 0.05) / (l1 + 0.05)

    #print(contrast_ratio, min_intensity, max_intensity)
    return contrast_ratio

# Step 1: Load the CSV file
csv_file = 'pixels.csv'  # Update this path to your CSV file
data = pd.read_csv(csv_file)

# Step 2: Extract pixel data from the CSV
# Extract the pixel values, starting from the second row
X = []

# Process each row in the data, starting from the first row after the header
for index, row in data.iterrows():
    # Split the space-separated string into integers
    pixel_values = np.fromstring(row['Pixels'], sep=' ', dtype=int)
    X.append(pixel_values)

# Convert the list of numpy arrays into a 2D numpy array
X = np.array(X)

# Verify the shape of X
print("Original shape of pixel data:", X.shape)

# Step 3: Reshape the data
image_size = (48,48)
num_images = X.shape[0]

# Check that pixel count for each image is correct
expected_pixel_count = image_size[0] * image_size[1]
if any(arr.size != expected_pixel_count for arr in X):
    raise ValueError("Not all images have the expected number of pixels.")

# Reshape the data
X = X.reshape(num_images, expected_pixel_count)  # Shape: (num_images, 256)

# Step 4: Automatically generate labels based on contrast
y = []
for i in range(num_images):
    # Convert the flattened pixel data back to an image for contrast calculation
    image = Image.fromarray(X[i].reshape(image_size).astype('uint8'))
    contrast_ratio = calculate_contrast(image)
    
    # Label as 1 (accessible) if contrast meets the threshold, else 0
    label = 1 if contrast_ratio >= CONTRAST_THRESHOLD else 0
    y.append(label)

y = np.array(y)

# Step 5: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=12)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
X_flattened = X_resampled.reshape(len(X_resampled), -1)  # Reshape to (num_samples, height*width)

# Step 6: Train the MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(128,64), activation='relu')

mlp.fit(X_flattened, y_resampled)

# Step 7: Evaluate the Model
y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Generate classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)


Original shape of pixel data: (768, 2304)
1.4184322033898304 176 255
2.656746031746032 88 255
1.5320366132723113 162 255
1.3270564915758176 189 255
2.9821826280623616 77 255
3.5329815303430085 63 255
1.1121262458471761 228 255
1.2584586466165413 200 255
1.1807760141093473 214 255
1.1755926251097453 215 255
1.2525724976613657 201 255
1.10752688172043 229 255
3.580213903743316 62 255
1.3336653386454183 188 255
3.0157657657657664 76 255
1.5233219567690557 163 255
1.4109589041095891 177 255
2.6306483300589396 89 255
1.6551297898640296 149 255
1.5408515535097813 161 255
1.4259850905218316 175 255
3.6785714285714293 60 255
3.0852534562211984 74 255
4.404605263157896 48 255
1.165361183637946 217 255
1.2409638554216869 203 255
1.2467411545623837 202 255
1.1704545454545454 216 255
4.333333333333333 49 255
3.050113895216401 75 255
3.628726287262873 61 255
1.4336188436830835 174 255
1.5497685185185184 160 255
1.6654228855721391 148 255
1.514705882352941 164 255
1.4649890590809629 170 255
1.567915